In [4]:
import pandas as pd
import numpy as np
from scipy.linalg import inv
from PortOpt_factor.optimizer import pyport
from ipca_utils import IPCA_impute

### Preprocess 

In [5]:
fn = "data/factor_data_new.p"
df = pd.read_pickle(fn)
pd.set_option("display.max_column", None)
df.head()

,id,eom,isin,cusip,sedol,gics,sic,naics,excntry,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,100100001,1970-09-30,None,None,None,NaN,NaN,NaN,USA,0.059230,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.389696,0.333709,1.095773,NaN,0.077817,-0.613949,NaN,NaN,NaN,8.27381,4.052973,0.147433,1.399905,-0.425303,1.022609,0.525045,0.284973,16.828861,-0.021824,3.848362,2.787572,2.82337,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.08338,-0.214719,NaN,0.032843,0.25101,0.266091,7.349377,3.858437,NaN,NaN,0.25101,0.060123,NaN,0.052671,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.83823,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,0.335870,NaN,NaN,1.129423,NaN,NaN,117,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.38813,-0.166415,NaN,NaN,NaN,NaN,NaN,NaN
1,100100001,1970-10-31,None,None,None,NaN,NaN,NaN,USA,-0.284600,-0.280000,0.063830,0.059230,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.366314,0.313686,1.030027,NaN,0.080072,-0.577112,NaN,NaN,NaN,8.27381,4.052973,0.140456,1.288395,-0.399785,0.961253,0.500198,0.267874,16.979129,-0.021824,3.848362,2.787572,2.82337,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.08338,-0.214719,NaN,0.032843,0.25101,0.266091,7.349377,3.858437,NaN,NaN,0.25101,NaN,NaN,NaN,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.83823,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,-0.071796,NaN,NaN,1.129423,NaN,NaN,118,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.38813,-0.166415,0.557225,0.007569,-0.131257,-0.242119,1.131001,-1.038514
2,100100001,1970-11-30,None,None,None,NaN,NaN,NaN,USA,0.106911,0.111111,-0.280000,-0.284600,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.508769,0.43567

## Dropped unused columns

In [6]:
cols_to_drop = ["isin", "cusip", "sedol", "gics", "sic", "naics", "excntry", "ret_exc_lead1m", "ret_exc", 'ret_local']
df_ipca = df.drop(cols_to_drop, axis=1)
# remove all rows/stocks that have missing values for next month return
df_ipca = df_ipca.dropna(subset=['ret_local_lead1m'])
df_ipca.head()

,id,eom,ret_local_lead1m,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,100100001,1970-09-30,0.063830,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.389696,0.333709,1.095773,NaN,0.077817,-0.613949,NaN,NaN,NaN,8.27381,4.052973,0.147433,1.399905,-0.425303,1.022609,0.525045,0.284973,16.828861,-0.021824,3.848362,2.787572,2.82337,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.08338,-0.214719,NaN,0.032843,0.25101,0.266091,7.349377,3.858437,NaN,NaN,0.25101,0.060123,NaN,0.052671,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.83823,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,0.335870,NaN,NaN,1.129423,NaN,NaN,117,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.38813,-0.166415,NaN,NaN,NaN,NaN,NaN,NaN
1,100100001,1970-10-31,-0.280000,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.366314,0.313686,1.030027,NaN,0.080072,-0.577112,NaN,NaN,NaN,8.27381,4.052973,0.140456,1.288395,-0.399785,0.961253,0.500198,0.267874,16.979129,-0.021824,3.848362,2.787572,2.82337,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.08338,-0.214719,NaN,0.032843,0.25101,0.266091,7.349377,3.858437,NaN,NaN,0.25101,NaN,NaN,NaN,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.83823,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,-0.071796,NaN,NaN,1.129423,NaN,NaN,118,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.38813,-0.166415,0.557225,0.007569,-0.131257,-0.242119,1.131001,-1.038514
2,100100001,1970-11-30,0.111111,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.508769,0.435675,1.430593,NaN,0.111211,-0.801545,NaN,NaN,NaN,8.27381,4.052973,0.180267,1.789437,-0.555257,1.335073,0.641973,0.372048,16.063606,-0.021824,3.848362,2.787572,2.82337,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.

In [7]:
len(df_ipca)

2641503

## IPCA Parameters

In [51]:
from logger import ErrorLogger
from tqdm import tqdm
logger = ErrorLogger()

In [13]:
from typing import List 

def IPCA_impute_w_mean(
    df: pd.DataFrame, 
    characteristics: List[str], 
    threshold: int = None,
    drop_characteristics: bool = True
):
    """
    Impute missing values for window data used in IPCA_factor

    Parameters
    ----------
    df:
        Pandas dataframe to impute
    characteristics:
        List of the characteristics
    threshold:
        Threshold (in percent) of missing data to drop a characteristics
    drop_characteristics:
        Bool indicator for dropping characteristics when imputing. Default True.
    
    """
    assert len(characteristics) > 0, "List of characteristics cannot be empty"

    assert not drop_characteristics or (drop_characteristics and threshold is not None), \
                        "When drop_characteristics is True, a threshold must be provided."


    # if drop_characteristics: remove all columns/characteristics 
    # that have missing values greater than the threshold
    if drop_characteristics:
        percent_missing = df.isnull().mean() * 100
        characteristics_to_drop = percent_missing[percent_missing > threshold].index
        df = df.drop(columns=characteristics_to_drop)
        characteristics_to_keep = [item for item in characteristics if item not in characteristics_to_drop]
        characteristics = characteristics_to_keep

    # fill the missing values at t with median across stock
    df_temp = df.set_index(['eom', 'id'])
    medians_by_time = df_temp.groupby(level=0).transform('median')
    df_imputed = df_temp.fillna(medians_by_time).reset_index()
    df_imputed_sorted = df_imputed.sort_values(by='eom')

    return df_imputed_sorted, characteristics

In [43]:
from ipca import InstrumentedPCA
def IPCA_factor_test(df, step, window_dates, current_date, characteristics, K, logger):

    # Filter the data to include only rows within the selected date range
    window_data = df[df['eom'].isin(window_dates)]
    window_data_imputed, characteristics_to_keep = IPCA_impute_w_mean(window_data, characteristics, threshold=50)
    
    # Get the last date data from the window data
    last_win_data = window_data_imputed[window_data_imputed['eom'] == window_dates[-1]]
    last_ids = last_win_data['id'].unique()
    

    # Get the predict/next data but only keep stocks & characteristics appeared in last data
    ft_data = df[df['eom'] == current_date]
    
    # only use characteristics that appear in the train data
    ft_data_aligned = ft_data[last_win_data.columns]
    ft_data_imputed, _ = IPCA_impute_w_mean(ft_data_aligned, characteristics_to_keep, drop_characteristics=False)
    

    y_next = ft_data_imputed['ret_local_lead1m']
    X_next = ft_data_imputed[characteristics_to_keep]
    X_last = last_win_data[characteristics_to_keep]
    
    assert X_next.shape[1] == X_last.shape[1], "Charateristics used for prediction has to be the same\
                    as the characteristics used for training"
    
    # IPCA model
    window_data_imputed.set_index(['id', 'eom'], inplace=True)
    y = window_data_imputed['ret_local_lead1m'] #lead return
    X = window_data_imputed[characteristics_to_keep]
    
  
    def get_ipca(X, y, K, step, date):
        while K >= 2:
            
            try:
                regr = InstrumentedPCA(n_factors=K, intercept=False, max_iter=500, iter_tol=1e-4, n_jobs= -1)
                regr = regr.fit(X=X, y=y, quiet = True)
                Gamma, Factors = regr.get_factors(label_ind=True)
                return Gamma, Factors
            except Exception as e:
                logger.log_error(f'ValueError for ({step}, K={K}): {date}', e)
                K -= 1
        
        logger.log_error(f'ValueError for {step}: {date}', 'Unable to fit the model with K >= 2')
        raise ValueError("Unable to fit the model")
    
    try:
        Gamma, Factors = get_ipca(X, y, K, step, cur_date)
    except Exception as e:
        raise
    
    return Gamma, Factors, y_next, X_next, X_last, last_ids

In [49]:
window_size = 180 #rolling window size
K = 6 #num of principle components
charateristics = df_ipca.columns[3:] #list of characteristics
unique_dates = sorted(df_ipca['eom'].unique()) #unique dates
T = len(unique_dates)

print (f'There are {T} unique days starting with {unique_dates[0]}')
print (f'Total {len(charateristics)} charateristics starting with {charateristics[0]}')

There are 649 unique days starting with 1969-12-31
Total 153 charateristics starting with niq_su


In [52]:
np.random.seed(42)

start = window_size
for t in range(start, T): 
    
    window_dates = unique_dates[t-window_size:t]
    cur_date = unique_dates[t]
    print (f'====Progress: {t}: {cur_date}====')

    # calculate ipca
    try:
        Gamma, factors, y_next, X_next, X_last, _ = IPCA_factor_test(df_ipca, t, window_dates, cur_date, charateristics, K, logger)
    except Exception as e:
        raise
    
    V_t = inv(Gamma.T @ X_last.T @ X_last @ Gamma) @ Gamma.T @ X_last.T
    Sigma_t= np.cov(factors, rowvar = True)
    mu_t = np.array(np.mean(factors, axis=1))

    # Grid search? grid search for regularization terms
    g1 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))
    g2 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))
    total_iterations = len(g1) * len(g2)
    max_ret_t = -1
    best_l1, best_l2 = None, None
  
    with tqdm(total=total_iterations) as pbar:
        for l1 in g1:
            for l2 in g2:
                OptimalPortfolioWeights_t = pyport.portfolio_optimization(
                    meanVec=np.array(mu_t),
                    sigMat=np.array(Sigma_t),
                    retTarget=0,
                    longShort=0.2,
                    maxAlloc=0.08,
                    lambda_l1=l1,
                    lambda_l2=l2,
                    riskfree=0,
                    assetsOrder=None,
                    maxShar=1,
                    factor=np.array(V_t.T),
                    turnover=None,
                    w_pre=None,
                    individual=False,
                    exposure_constrain=0,
                    w_bench=None,
                    factor_exposure_constrain=None,
                    U_factor=None,
                    general_linear_constrain=None,
                    U_genlinear=0,
                    w_general=None,
                    TE_constrain=0,
                    general_quad=0,
                    Q_w=None,
                    Q_b=None,
                    Q_bench=None
                )[0].reshape(-1, 1)
        
                ret_t = (inv(Gamma.T @ X_next.T @ X_next @ Gamma) @ Gamma.T @ X_next.T @ y_next).T @ OptimalPortfolioWeights_t
                if ret_t > max_ret_t:
                    max_ret_t = ret_t
                    best_l1, best_l2 = l1, l2
                
                pbar.update(1)
                
    print (t, max_ret_t, best_l1, best_l2)
    break
    
    

====Progress: 180: 1984-12-31====


The panel dimensions are:
n_samples: 4373 , L: 116 , T: 158


[========================================================================] 100%
100%|██████████| 100/100 [08:23<00:00,  5.03s/it]

180 [0.24644166] 0.9008241153272052 0.0009491175455796851
